## **Fact Orders**

### **Data Reading**

In [0]:
df = spark.sql("select * from dec25etlproject.silver.orders_silver")
display(df)

In [0]:
df_dimcus = spark.sql("select DimCustomerkey, customer_id as dim_custome_id from dec25etlproject.gold.dimcustomers")

df_dimpro = spark.sql("select product_id as DimProductkey,  product_id as dim_product_id from dec25etlproject.gold.dimproducts")

## **Fact DataFrame**

In [0]:
df_fact = df.join(df_dimcus, df['customer_id'] == df_dimcus['dim_custome_id'], how='left').join(df_dimpro, df["product_id"] == df_dimpro["dim_product_id"], how='left')

df_fact_new = df_fact.drop('dim_custome_id', 'dim_product_id', 'customer_id', 'product_id')

In [0]:
df_fact_new.display()

In [0]:
df_fact_new = df_fact_new.orderBy("order_id")
df_fact_new.display()

## **Upsert on Fact Table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("dec25etlproject.gold.Factorders"):
  
    dlt_obj = DeltaTable.forName(spark, "dec25etlproject.gold.Factorders")
  
    dlt_obj.alias("trg").merge(df_fact_new.alias("src"), "trg.order_id = src.order_id AND trg.DimCustomerkey = src.DimCustomerkey AND trg.DimProductkey = src.DimProductkey")\
      .whenMatchedUpdateAll()\
      .whenNotMatchedInsertAll()\
      .execute()

else:
    df_fact_new.write.format("delta")\
      .option("path", "abfss://gold@sadec25etlproject.dfs.core.windows.net/Factorders")\
      .saveAsTable("dec25etlproject.gold.Factorders")

In [0]:
%sql
select * from dec25etlproject.gold.factorders;